<a href="https://colab.research.google.com/github/nhibb262/-ISYS574-ML-Group-Project/blob/main/Notebook/02_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 - Data Cleaning Pipeline

**Author:** [Your Name]  
**Date:** [YYYY-MM-DD]  
**Purpose:** Clean and prepare the Our415 dataset for modeling

---

## Table of Contents
1. [Setup & Load Data](#1-setup--load-data)
2. [Cleaning Step 1: Remove Duplicates](#2-cleaning-step-1-remove-duplicates)
3. [Cleaning Step 2: Handle Missing Values](#3-cleaning-step-2-handle-missing-values)
4. [Cleaning Step 3: Standardize Text](#4-cleaning-step-3-standardize-text)
5. [Cleaning Step 4: Create Derived Columns](#5-cleaning-step-4-create-derived-columns)
6. [Cleaning Step 5: Data Type Conversions](#6-cleaning-step-5-data-type-conversions)
7. [Final Validation](#7-final-validation)
8. [Save Cleaned Data](#8-save-cleaned-data)
9. [Cleaning Summary](#9-cleaning-summary)

---

## Change Log
| Date | Change | Author |
|------|--------|--------|
| YYYY-MM-DD | Initial version | [Name] |

---

## Cleaning Philosophy

> **Document every decision.** For each cleaning step, explain:
> 1. What was the problem?
> 2. What options did we consider?
> 3. What did we choose and why?
> 4. How many records were affected?

## 1. Setup & Load Data

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project paths
import os
PROJECT_PATH = '/content/drive/MyDrive/sf-events-explorer'
RAW_DATA_PATH = f'{PROJECT_PATH}/data/raw'
PROCESSED_DATA_PATH = f'{PROJECT_PATH}/data/processed'

Mounted at /content/drive


In [2]:
# Imports
import pandas as pd
import numpy as np
import re
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
# Load raw data
df = pd.read_csv(f'{RAW_DATA_PATH}/Our415_Events_and_Activities.csv')
initial_count = len(df)
print(f"Loaded {initial_count:,} records")

# Track changes
cleaning_log = []

Loaded 1,874 records


In [4]:
# Helper function to log cleaning steps
def log_step(step_name, before_count, after_count, notes=""):
    removed = before_count - after_count
    cleaning_log.append({
        'step': step_name,
        'before': before_count,
        'after': after_count,
        'removed': removed,
        'removed_pct': round(removed / before_count * 100, 2) if before_count > 0 else 0,
        'notes': notes
    })
    print(f"[{step_name}] {before_count:,} → {after_count:,} ({removed:,} removed, {removed/before_count*100:.1f}%)")
    if notes:
        print(f"   Notes: {notes}")

## 2. Cleaning Step 1: Remove Duplicates

### Decision Documentation
- **Problem:** [Describe duplicate issue from audit]
- **Options considered:**
  1. Remove exact duplicates only
  2. Remove based on key columns (event_name + location + date)
  3. Keep all (if duplicates are intentional, e.g., recurring events)
- **Decision:** [What we chose]
- **Rationale:** [Why]

In [5]:
# Check for exact duplicates
before = len(df)
exact_dupes = df.duplicated().sum()
print(f"Exact duplicates found: {exact_dupes}")

Exact duplicates found: 0


In [6]:
# Remove exact duplicates
df = df.drop_duplicates()
log_step("Remove exact duplicates", before, len(df), f"Removed {exact_dupes} exact duplicate rows")

[Remove exact duplicates] 1,874 → 1,874 (0 removed, 0.0%)
   Notes: Removed 0 exact duplicate rows


In [7]:
# Check for duplicates based on key columns (optional)
# Uncomment and modify if needed

# key_cols = ['event_name', 'site_location_name', 'event_start_date']
# before = len(df)
# df = df.drop_duplicates(subset=key_cols, keep='first')
# log_step("Remove key column duplicates", before, len(df), f"Based on {key_cols}")

## 3. Cleaning Step 2: Handle Missing Values

### Decision Documentation
- **Columns with missing values:** [List from audit]
- **Strategy per column:**

| Column | Missing % | Strategy | Rationale |
|--------|-----------|----------|----------|
| admission_price | ~99% | Drop column | Not useful for our use case |
| event_description | ~X% | Keep nulls | Can search by name only |
| ... | ... | ... | ... |

In [8]:
# Review missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'missing_count': missing, 'missing_pct': missing_pct})
missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)

,missing_count,missing_pct
admission_price,1874,100.00
special_needs_eligibility_tags,1856,99.04
race_ethnicity_eligibility_tags,1852,98.83
sogi_eligibility_tags,1796,95.84
language_eligibility_tags,1729,92.26
site_email,1102,58.80
days_of_week,848,45.25
fee,139,7.42
age_group_eligibility_tags,120,6.40
supervisor_district,105,5.60


In [9]:
# Drop columns with very high missing values (>90%)
# DOCUMENT: List columns being dropped and why

cols_to_drop = []  # Add column names here, e.g., ['admission_price', 'column2']

# Check which columns have >90% missing
high_missing_cols = missing_df[missing_df['missing_pct'] > 90].index.tolist()
print(f"Columns with >90% missing: {high_missing_cols}")

# Decide which to drop
# cols_to_drop = high_missing_cols  # Uncomment to drop all high-missing columns

Columns with >90% missing: ['admission_price', 'race_ethnicity_eligibility_tags', 'special_needs_eligibility_tags', 'sogi_eligibility_tags', 'language_eligibility_tags']


In [10]:
# Drop selected columns
if cols_to_drop:
    before_cols = len(df.columns)
    df = df.drop(columns=cols_to_drop, errors='ignore')
    print(f"Dropped {before_cols - len(df.columns)} columns: {cols_to_drop}")

In [11]:
# Handle missing values in critical columns
# DOCUMENT: Explain strategy for each column

# Example: Fill missing descriptions with event name
if 'event_description' in df.columns and 'event_name' in df.columns:
    before_nulls = df['event_description'].isnull().sum()
    df['event_description'] = df['event_description'].fillna(df['event_name'])
    after_nulls = df['event_description'].isnull().sum()
    print(f"event_description: filled {before_nulls - after_nulls} nulls with event_name")

event_description: filled 61 nulls with event_name


In [12]:
# Drop rows with missing critical fields (if any)
critical_cols = ['event_name']  # Must have these

before = len(df)
df = df.dropna(subset=critical_cols)
log_step("Drop rows missing critical fields", before, len(df), f"Critical cols: {critical_cols}")

[Drop rows missing critical fields] 1,874 → 1,874 (0 removed, 0.0%)
   Notes: Critical cols: ['event_name']


## 4. Cleaning Step 3: Standardize Text

### Decision Documentation
- **Text columns:** event_name, event_description
- **Standardization steps:**
  1. Strip whitespace
  2. Remove extra spaces
  3. [Add more as needed]

In [13]:
# Text cleaning function
def clean_text(text):
    """Clean text fields: strip whitespace, remove extra spaces."""
    if pd.isna(text):
        return text
    text = str(text)
    text = text.strip()  # Remove leading/trailing whitespace
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text

In [14]:
# Apply text cleaning
text_cols = ['event_name', 'event_description']

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)
        print(f"Cleaned text in: {col}")

Cleaned text in: event_name
Cleaned text in: event_description


In [15]:
# Standardize category names (if needed)
if 'events_category' in df.columns:
    # Check unique values
    print("Category values before standardization:")
    print(df['events_category'].value_counts())

    # Apply any necessary mappings
    # category_mapping = {
    #     'Old Name': 'New Name',
    # }
    # df['events_category'] = df['events_category'].replace(category_mapping)

Category values before standardization:
events_category
Sports & Recreation    1026
Library Events          709
Events for Ages 0-5     139
Name: count, dtype: int64


## 5. Cleaning Step 4: Create Derived Columns

### Decision Documentation
- **New columns created:**
  1. `search_text`: Combined event_name + event_description + category for TF-IDF
  2. `is_free`: Boolean based on fee column
  3. [Add more as needed]

In [16]:
# Create search_text column (critical for TF-IDF)
text_columns = ['event_name', 'event_description', 'events_category']
existing_text_cols = [col for col in text_columns if col in df.columns]

df['search_text'] = df[existing_text_cols].fillna('').agg(' '.join, axis=1)
df['search_text'] = df['search_text'].apply(clean_text)

print(f"Created search_text column from: {existing_text_cols}")
print(f"Sample: {df['search_text'].iloc[0][:200]}...")

Created search_text column from: ['event_name', 'event_description', 'events_category']
Sample: Learn to Swim - Level 1 This class is designed to orient participants to the aquatic environment and to help them gain basic aquatic skills. In addition, participants start learning about how to be sa...


In [17]:
# Create is_free boolean
if 'fee' in df.columns:
    # Check current values
    print(f"Fee column values: {df['fee'].unique()}")

    # Create boolean (adjust logic based on actual values)
    df['is_free'] = df['fee'].apply(lambda x: str(x).lower() in ['false', 'no', '0', 'nan', ''])
    print(f"is_free distribution: {df['is_free'].value_counts().to_dict()}")

Fee column values: [True False nan]
is_free distribution: {True: 1074, False: 800}


In [18]:
# Create time_of_day categories (optional)
if 'start_time' in df.columns:
    def categorize_time(time_str):
        """Categorize time into morning/afternoon/evening."""
        if pd.isna(time_str):
            return 'unknown'
        try:
            hour = int(str(time_str).split(':')[0])
            if hour < 12:
                return 'morning'
            elif hour < 17:
                return 'afternoon'
            else:
                return 'evening'
        except:
            return 'unknown'

    df['time_of_day'] = df['start_time'].apply(categorize_time)
    print(f"time_of_day distribution: {df['time_of_day'].value_counts().to_dict()}")

time_of_day distribution: {'morning': 836, 'afternoon': 724, 'evening': 314}


## 6. Cleaning Step 5: Data Type Conversions

### Decision Documentation
- **Columns to convert:** [List columns and target types]

In [19]:
# Check current data types
print("Current data types:")
print(df.dtypes)

Current data types:
id                                  object
org_name                            object
event_name                          object
event_description                   object
event_start_date                    object
event_end_date                      object
days_of_week                        object
start_time                          object
end_time                            object
more_info                           object
fee                                 object
admission_price                    float64
site_location_name                  object
event_photo                         object
events_category                     object
age_group_eligibility_tags          object
race_ethnicity_eligibility_tags     object
special_needs_eligibility_tags      object
sogi_eligibility_tags               object
language_eligibility_tags           object
site_address                        object
site_phone                          object
site_email                        

In [20]:
# Convert date columns (if needed)
date_cols = ['event_start_date', 'event_end_date']

for col in date_cols:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            print(f"Converted {col} to datetime")
        except Exception as e:
            print(f"Could not convert {col}: {e}")

Converted event_start_date to datetime
Converted event_end_date to datetime


/tmp/ipython-input-2716257554.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/tmp/ipython-input-2716257554.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [21]:
# Convert numeric columns (if needed)
numeric_cols = ['latitude', 'longitude']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"Converted {col} to numeric")

Converted latitude to numeric
Converted longitude to numeric


## 7. Final Validation

In [22]:
# Final data quality checks
print("FINAL DATA QUALITY REPORT")
print("=" * 50)
print(f"Total records: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"\nRecords removed during cleaning: {initial_count - len(df):,} ({(initial_count - len(df))/initial_count*100:.1f}%)")

FINAL DATA QUALITY REPORT
Total records: 1,874
Total columns: 33

Records removed during cleaning: 0 (0.0%)


In [23]:
# Check for remaining nulls in critical columns
critical_cols = ['event_name', 'search_text']
print("\nNull check for critical columns:")
for col in critical_cols:
    if col in df.columns:
        null_count = df[col].isnull().sum()
        print(f"  {col}: {null_count} nulls")


Null check for critical columns:
  event_name: 0 nulls
  search_text: 0 nulls


In [24]:
# Verify search_text is populated
empty_search = (df['search_text'].str.strip() == '').sum()
print(f"\nEmpty search_text fields: {empty_search}")

if empty_search > 0:
    print("WARNING: Some records have empty search_text!")


Empty search_text fields: 0


In [25]:
# Preview cleaned data
print("\nSAMPLE OF CLEANED DATA:")
df[['event_name', 'events_category', 'search_text']].head(3)


SAMPLE OF CLEANED DATA:


,event_name,events_category,search_text
0,Learn to Swim - Level 1,Sports & Recreation,Learn to Swim - Level 1 This class is designed to orient participants to the aquatic environment...
1,Jr. Warriors Basketball - Herz - 5th/6thGrade,Sports & Recreation,Jr. Warriors Basketball - Herz - 5th/6thGrade Welcome to our 25th Annual SFRPD All-City Jr. Warr...
2,Learn to Swim - Level 4,Sports & Recreation,Learn to Swim - Level 4 This class should improve participants' proficiency in performing the sw...


## 8. Save Cleaned Data

In [26]:
# Save cleaned dataset
output_path = f'{PROCESSED_DATA_PATH}/events_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"Saved cleaned data to: {output_path}")
print(f"Final size: {len(df):,} records")

Saved cleaned data to: /content/drive/MyDrive/sf-events-explorer/data/processed/events_cleaned.csv
Final size: 1,874 records


In [27]:
# Save cleaning log
cleaning_log_df = pd.DataFrame(cleaning_log)
log_path = f'{PROCESSED_DATA_PATH}/cleaning_log.csv'
cleaning_log_df.to_csv(log_path, index=False)
print(f"Saved cleaning log to: {log_path}")
cleaning_log_df

Saved cleaning log to: /content/drive/MyDrive/sf-events-explorer/data/processed/cleaning_log.csv


,step,before,after,removed,removed_pct,notes
0,Remove exact duplicates,1874,1874,0,0.0,Removed 0 exact duplicate rows
1,Drop rows missing critical fields,1874,1874,0,0.0,Critical cols: ['event_name']


## 9. Cleaning Summary

### Before vs After
| Metric | Before | After |
|--------|--------|-------|
| Total Records | [fill in] | [fill in] |
| Total Columns | [fill in] | [fill in] |

### Cleaning Steps Applied
1. **Duplicates:** [X] removed
2. **Missing values:** [describe handling]
3. **Text standardization:** [describe]
4. **New columns:** search_text, is_free, time_of_day

### Columns in Final Dataset
[List final columns]

### Ready for Modeling
The cleaned dataset is ready for:
- TF-IDF vectorization using `search_text` column
- Rule-based feature extraction using `events_category`, `age_group_eligibility_tags`, etc.
- Model evaluation and comparison

In [28]:
# List final columns
print("FINAL COLUMNS:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

FINAL COLUMNS:
 1. id
 2. org_name
 3. event_name
 4. event_description
 5. event_start_date
 6. event_end_date
 7. days_of_week
 8. start_time
 9. end_time
10. more_info
11. fee
12. admission_price
13. site_location_name
14. event_photo
15. events_category
16. age_group_eligibility_tags
17. race_ethnicity_eligibility_tags
18. special_needs_eligibility_tags
19. sogi_eligibility_tags
20. language_eligibility_tags
21. site_address
22. site_phone
23. site_email
24. latitude
25. longitude
26. point
27. analysis_neighborhood
28. supervisor_district
29. data_as_of
30. data_loaded_at
31. search_text
32. is_free
33. time_of_day
